# requests

Requests is an elegant and simple HTTP library for Python, built for human beings.

requests - стороняя библиотека - она не входит в стандартный пакет питона, поэтому воспользуемся менеджером пакетов `pip3` чтобы ее установить

In [47]:
!pip3 install requests

Подключаем библиотеку

In [ ]:
import requests

Отправляем `get` запрос на `google.com`

In [ ]:
response = requests.get('https://google.com')

Функции пакета `requests` возвращают объект класса `Response`, нас будут интересовать такие аттрибуты как `text`, `status_code` и, иногда, метод `json()`

In [ ]:
response.text[:100]

In [ ]:
response.status_code

Что позволяет делать requests? Если вкратце - роботизировать посещение сети интернет  
Примеры задач которые приходят в голову первыми:
 - Сбор данных в открытых источниках (скачиваем страницу, парсим, сохраняем в файлик)
 - __Автоматизация__
 - Работа с апи сайтов
 - Тестирование / Тестирование безопасности

## Сбор данных

In [48]:
url = 'https://accounts.tsu.ru/Profile/Index/108023'

resp = requests.get(url).text

In [49]:
resp[:100]

'<!DOCTYPE html>\r\n<html lang="ru-RU">\r\n<head>\r\n    <meta http-equiv="Content-Type" content="text/html'

In [50]:
import re # На самом деле здесь лучше использовать BeatufilSoup, но мы легких путей не ищем!

fullname = re.search('alt="(.*?)"', resp).group(1)
print(fullname)

Палкин Роман Владимирович


Конечно это можно было сделать и руками, но используя requests ваши _руки_ не будут знать усталости!

In [51]:
base_url = 'https://accounts.tsu.ru/Profile/Index/'

fullnames = []

for i in range(52583, 52783): # Диапозон уменьшен в целях экономии времени
    if not i % 10: # печатать прогресс каждые 100 запросов
        print(i)
    try: # Когда работаете со сторонними ресурсами / сетью, ВСЕГДА думайте об ошибках!
        resp = requests.get(base_url + str(i)).text
        fullname = re.search('alt="(.*)"', resp).group(1)
        fullnames.append(fullname)
        print(fullname)
    except:
        pass

Кригер Ольга Сергеевна
Митрофанова Кристина Игоревна
Буянова Кристина Сергеевна
Саакян Аник Усиковна
Ширяева Елизавета Александровна
ВКонтакте
Марченко Лидия 
52590
Зайцев Александр Александрович
Шимко Остап Витальевич
Яровиков Александр 
Диденко Дарья 
Жукова Екатерина 
Терешкина Людмила 
Пахатинская Валентина Юрьевна
Кучина Виктория Константиновна
Мороз Александра Дмитриевна
Павлов Егор Вячеславович
52600
Сергиенко Виктория Александровна
Кудрявцева Елизавета Владимировна
Семенов Вячеслав Витальевич
Кеслер Виталий Дмитриевич
Трабаев Тимур Даулетович
Федченко Артём Сергеевич
Арбатская Юлия Яковлевна
Грузин Тимофей Евгеньевич
Зайцева Анна 
Шевляков Артем Игоревич
52610
Талгатов Эльдар Талгатович
Шайкенов Роман Бахытбекович
Нурмагамбетов Шарип Кайратович
Шейко Анна Юрьевна
Магомедова Мариям Садулаевна
Ким Михаил 
Сивак Дарья 
Махметов Руслан Маратович
Никуленкин Никита Сергеевич
Буховец Егор Александрович
52620
Джуманов Артур Тимурович
Сохорев Кирилл Константинович
Носонов Евгений Фёдоро

In [52]:
len(fullnames)

197

Используя асинхронное программирование и доступные любому ребенку впн и прокси, гигабайты таких данных можно собирать за считанные часы

## Автоматизация

Чаще всего, веб-сайты используют какое-либо браузерное хранилище, для того чтобы хранить данные о текущей сессии пользователя (cookies, app storage etc.), поэтому для автоматизации часто нужно работать в пределах одной сессии

In [53]:
session = requests.Session()

In [54]:
session.get('http://92.63.70.135')

<Response [200]>

In [55]:
dict(session.cookies)

{'session': 'eyJtb25leSI6MH0.XeuWPw.77Q58WwIT1eiD_Td1f2Jcrm88LA'}

In [56]:
for i in range(1000):
    session.post('http://92.63.70.135/click/')

In [57]:
resp = session.post('http://92.63.70.135/buy/')
print(resp.text)

Secret value is 42



## Работа с API

Токен можно получить по ссылке:

https://oauth.vk.com/authorize?client_id=5440699&display=page&redirect_uri=vk.com/callback&response_type=token&v=5.80

Токен - секретный ключ для вашего аккаунта. Его нужно передавать в большинстве методов VK API

Запрос к VK API - GET или POST HTTP запрос вида:

`https://api.vk.com/method/METHOD_NAME?PARAMETERS&access_token=ACCESS_TOKEN&v=5.80`

Где ACCESS_TOKEN - ваш токен, METHOD_NAME - имя метода, PARAMETERS - его параметры, а v - версия VK API (можно оставить 5.80)

Список методов и параметров можно посмотреть здесь - https://vk.com/dev/methods

**Внимание**

Если вы запускаете этот ноутбук в jupyter hub, а не в своем локальном jupyter notebook, полученный таким образом токен нельзя будет использовать на удаленных серверах (таким является наш jupyter hub)

In [58]:
token = ''

resp = requests.post('https://api.vk.com/method/users.get', data={'user_ids': '1',
                                                                  'access_token': token})

In [60]:
print(resp.json())

{'error': {'error_code': 5, 'error_msg': 'User authorization failed: access_token was given to another ip address.', 'request_params': [{'key': 'method', 'value': 'users.get'}, {'key': 'oauth', 'value': '1'}, {'key': 'user_ids', 'value': '1'}]}}
